<a href="https://colab.research.google.com/github/fleanend/TorchGlocalK/blob/main/Glocal_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
# !git clone https://github.com/usydnlp/Glocal_K.git

In [182]:
from time import time
from scipy.sparse import csc_matrix
import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter

torch.manual_seed(1284)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data Loader Function

In [183]:
def load_data_100k(path='./', delimiter='\t'):

    train = np.loadtxt(path+'movielens_100k_u1.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'movielens_100k_u1.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:,0]).size  # num of users
    n_m = np.unique(total[:,1]).size  # num of movies
    n_train = train.shape[0]  # num of training ratings
    n_test = test.shape[0]  # num of test ratings

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_train):
        train_r[train[i,1]-1, train[i,0]-1] = train[i,2]

    for i in range(n_test):
        test_r[test[i,1]-1, test[i,0]-1] = test[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')    # 평점이 0인 경우를 1e-12로 처리?

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

# Load Data

In [184]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
data_path = ''
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [185]:
# Data Load
try:
    path = data_path + 'Glocal_K/data/MovieLens_100K/'
    print(path)
    n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(path=path, delimiter='\t')
except Exception:
    print('Error: Unable to load data')

Glocal_K/data/MovieLens_100K/
data matrix loaded
num of users: 943
num of movies: 1682
num of training ratings: 80000
num of test ratings: 20000


In [186]:
# Common hyperparameter settings
n_hid = 500 # size of hidden layers
n_dim = 5 # inner AE embedding size
n_layers = 2 # number of hidden layers
gk_size = 3 # width=height of kernel for convolution

# Hyperparameters to tune for specific case
max_epoch_p = 1000 # max number of epochs for pretraining
max_epoch_f = 1500 # max number of epochs for finetuning
patience_p = 5 # number of consecutive rounds of early stopping condition before actual stop for pretraining
patience_f = 10 # and finetuning
tol_p = 1e-4 # minimum threshold for the difference between consecutive values of train rmse, used for early stopping, for pretraining
tol_f = 1e-5 # and finetuning
lambda_2 = 20. # regularisation of number or parameters
lambda_s = 0.006 # regularisation of sparsity of the final matrix
dot_scale = 1 # dot product weight for global kernel

## side information 추가

In [187]:
import pandas as pd

In [188]:
### movielens u.item 읽어오기
headers = ['movie id' , 'movie title' , 'release date' , 'video release date' ,
              'IMDb URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western']
ml_item = pd.read_csv("./MovieLens-IMDB/movielens/raw/ml-100k/u.item", sep = '|', encoding = 'latin-1', names = headers)

ml_onehot = ml_item.iloc[:, 5:]
### genre를 숫자로 인코딩
genre_dict = {}
for i, genre in enumerate(headers[5:]):
    genre_dict[genre] = i

for idx, row in ml_item.iterrows():
    genre = row[5:]
    genre = genre[genre == 1].index[0]
    ml_item.loc[idx, 'genre'] = int(genre_dict[genre])
ml_label = ml_item[['genre']]

### movielens 100k - imdb 매칭 파일 읽어오기

In [189]:
import pandas as pd
ml_imdb_df = pd.read_csv('./MovieLens-IMDB/movielens/statistics/ml-100k/matched_title.pd', sep = '\t')
ml_imdb_df['imdb_id'] = ml_imdb_df['imdb_id'].apply(lambda x: x.split('/')[0]) # imdb id가 여러 개 인 경우 하나만 선택
ml_imdb_df

,ml_id,imdb_id
0,1,tt0114709
1,2,tt21638798
2,3,tt0113101
3,4,tt0113161
4,5,tt0112722
...,...,...
1672,1678,tt0119711
1673,1679,tt0120594
1674,1680,tt0120148
1675,1681,tt0111804


In [190]:
### imdb 정보 파일 읽어오기 (ml 데이터에 있는 것만 남김김)
imdb = pd.read_csv("./MovieLens-IMDB/IMDb/merged_imdb_data.csv")
imdb = imdb[imdb['title_id'].isin(ml_imdb_df['imdb_id'])]
imdb

/tmp/ipykernel_648409/2842006602.py:2: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv("./MovieLens-IMDB/IMDb/merged_imdb_data.csv")


,title_id,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,...,director_name,birthYear_x,deathYear_x,primaryProfession_x,knownForTitles_x,writer_name,birthYear_y,deathYear_y,primaryProfession_y,knownForTitles_y
47740,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",1.0,...,F.W. Murnau,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN
47741,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",10.0,...,F.W. Murnau,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN
47742,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",11.0,...,F.W. Murnau,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN
47743,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",12.0,...,F.W. Murnau,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN
47744,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",13.0,...,F.W. Murnau,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46245984,tt7979062,tvEpisode,Phenomenon,Phenomenon,0.0,1996.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153"
46245999,tt7979088,tvEpisode,2 Days in the Valley,2 Days in the Valley,0.0,1996.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153"
46246002,tt7979092,tvEpisode,Tomorrow Never Dies,Tomorrow Never Dies,0.0,1997.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153"
47122081,tt8387154,tvEpisode,Dingo,Dingo,0.0,1988.0,NaN,NaN,Animation,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
imdb['ml_id'] = imdb['title_id'].apply(lambda x: ml_imdb_df[ml_imdb_df['imdb_id'] == x]['ml_id'].values[0])
imdb

,title_id,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,...,birthYear_x,deathYear_x,primaryProfession_x,knownForTitles_x,writer_name,birthYear_y,deathYear_y,primaryProfession_y,knownForTitles_y,ml_id
47740,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",1.0,...,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN,675
47741,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",10.0,...,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN,675
47742,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",11.0,...,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN,675
47743,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",12.0,...,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN,675
47744,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",13.0,...,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN,675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46245984,tt7979062,tvEpisode,Phenomenon,Phenomenon,0.0,1996.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153",125
46245999,tt7979088,tvEpisode,2 Days in the Valley,2 Days in the Valley,0.0,1996.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153",1011
46246002,tt7979092,tvEpisode,Tomorrow Never Dies,Tomorrow Never Dies,0.0,1997.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153",751
47122081,tt8387154,tvEpisode,Dingo,Dingo,0.0,1988.0,NaN,NaN,Animation,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1346


### imdb 인코딩

In [192]:
# imdb.columns

In [193]:
print(len(imdb))
imdb.drop_duplicates(subset = 'ml_id', inplace = True)
print(len(imdb))

56073
1656


In [194]:
imdb

,title_id,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,...,birthYear_x,deathYear_x,primaryProfession_x,knownForTitles_x,writer_name,birthYear_y,deathYear_y,primaryProfession_y,knownForTitles_y,ml_id
47740,tt0013442,movie,Nosferatu: A Symphony of Horror,"Nosferatu, eine Symphonie des Grauens",0.0,1922.0,NaN,94.0,"Fantasy,Horror",1.0,...,1888.0,1931.0,"director,writer,producer","tt0018455,tt0013442,tt0022458,tt0012973",NaN,NaN,NaN,NaN,NaN,675
69023,tt0017350,movie,The Scarlet Letter,The Scarlet Letter,0.0,1926.0,NaN,115.0,Drama,1.0,...,1879.0,1960.0,"director,actor,writer","tt0050986,tt0012364,tt0008663,tt0014972",NaN,NaN,NaN,NaN,NaN,1542
93508,tt0020697,movie,The Blue Angel,Der blaue Engel,0.0,1930.0,NaN,104.0,"Drama,Music,Romance",1.0,...,1894.0,1969.0,"director,writer,assistant_director","tt0026276,tt0018839,tt0046712,tt0021156",NaN,NaN,NaN,NaN,NaN,617
103425,tt0022100,movie,M,M - Eine Stadt sucht einen Mörder,0.0,1931.0,NaN,99.0,"Crime,Mystery,Thriller",1.0,...,1890.0,1976.0,"director,writer,producer","tt0057345,tt0022100,tt0017136,tt0027652",NaN,NaN,NaN,NaN,NaN,656
109229,tt0022879,movie,A Farewell to Arms,A Farewell to Arms,0.0,1932.0,NaN,80.0,"Drama,Romance,War",1.0,...,1894.0,1962.0,"actor,director,producer","tt0018379,tt0021635,tt0025569,tt0016013",NaN,NaN,NaN,NaN,NaN,1124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46245984,tt7979062,tvEpisode,Phenomenon,Phenomenon,0.0,1996.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153",125
46245999,tt7979088,tvEpisode,2 Days in the Valley,2 Days in the Valley,0.0,1996.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153",1011
46246002,tt7979092,tvEpisode,Tomorrow Never Dies,Tomorrow Never Dies,0.0,1997.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,Annamarie Griffin,NaN,NaN,"producer,writer,miscellaneous","tt0429345,tt0386938,tt0096562,tt0197153",751
47122081,tt8387154,tvEpisode,Dingo,Dingo,0.0,1988.0,NaN,NaN,Animation,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1346


In [229]:
print(imdb.iloc[1])

title_id                                             tt0017350
titleType                                                movie
primaryTitle                                The Scarlet Letter
originalTitle                               The Scarlet Letter
isAdult                                                    0.0
startYear                                               1926.0
endYear                                                    NaN
runtimeMinutes                                           115.0
genres                                                   Drama
ordering                                                   1.0
title                                       The Scarlet Letter
region                                                     NaN
language                                                   NaN
types                                                 original
attributes                                                 NaN
isOriginalTitle                                        

In [195]:
imdb.columns

Index(['title_id', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres', 'ordering', 'title',
       'region', 'language', 'types', 'attributes', 'isOriginalTitle',
       'directors', 'writers', 'director_name', 'birthYear_x', 'deathYear_x',
       'primaryProfession_x', 'knownForTitles_x', 'writer_name', 'birthYear_y',
       'deathYear_y', 'primaryProfession_y', 'knownForTitles_y', 'ml_id'],
      dtype='object')

In [196]:
print(imdb['genres'].value_counts())

genres
Drama                           109
Drama,Romance                    93
Comedy,Drama,Romance             92
Comedy,Drama                     90
Comedy                           66
                               ... 
Comedy,Romance,War                1
Biography,Comedy,Documentary      1
Adventure,Comedy,Romance          1
Comedy,Crime,Music                1
News                              1
Name: count, Length: 287, dtype: int64


In [197]:
imdb_encode = imdb.copy()

In [198]:
### genre -> one-hot encoding
genre = imdb['genres'].str.get_dummies(sep=', ')

### directors, writers -> label encoding
from sklearn.preprocessing import LabelEncoder
director_encoder = LabelEncoder()
writer_encoder = LabelEncoder()
imdb_encode['director_id'] = director_encoder.fit_transform(imdb_encode['directors'])
imdb_encode['writer_id'] = writer_encoder.fit_transform(imdb_encode['writers'])

# ### genre -> label encoding
# from sklearn.preprocessing import LabelEncoder
# genre_encoder = LabelEncoder()
# genre = genre_encoder.fit_transform(imdb_encode['genres'])
# imdb_features = pd.DataFrame(genre, columns = ['genre'], index = imdb_encode['ml_id'])

### imdb 피처 만들기
imdb_features = pd.concat([genre, imdb_encode[['director_id', 'writer_id','ml_id']]], axis = 1).set_index('ml_id')

In [199]:
imdb_features

,Action,"Action,Adventure","Action,Adventure,Animation","Action,Adventure,Biography","Action,Adventure,Comedy","Action,Adventure,Crime","Action,Adventure,Drama","Action,Adventure,Family","Action,Adventure,Fantasy","Action,Adventure,Horror",...,"News,Talk-Show",Romance,Sci-Fi,"Sci-Fi,Short",Short,Sport,Thriller,Western,director_id,writer_id
ml_id,,,,,,,,,,,,,,,,,,,,,
675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,270,1315
1542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,947,751
617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1018,970
656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,66,1382
1124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,373,688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1107,1487
1011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1107,1487
751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1107,1487


In [200]:
imdb_features['features'] = imdb_features.apply(lambda x: x.values, axis = 1)
print(imdb_features['features'].apply(lambda x: len(x)).value_counts())

features
289    1656
Name: count, dtype: int64


/tmp/ipykernel_648409/1230459646.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  imdb_features['features'] = imdb_features.apply(lambda x: x.values, axis = 1)


In [201]:
# title을 bert로 인코딩
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

bert_encode = []
for i in tqdm(range(len(imdb_encode))):
    text = imdb_encode.iloc[i]['primaryTitle']
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)
        # 마지막 은닉 상태에서 CLS 토큰의 출력 가져오기
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS 토큰의 임베딩
    emb_array = embeddings.cpu().numpy().squeeze()  
    emb_array = list(emb_array)
    bert_encode.append(emb_array)
imdb_features['bert_encode'] = bert_encode
imdb_features

100%|██████████| 1656/1656 [00:33<00:00, 50.08it/s]
/tmp/ipykernel_648409/3320527983.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  imdb_features['bert_encode'] = bert_encode


,Action,"Action,Adventure","Action,Adventure,Animation","Action,Adventure,Biography","Action,Adventure,Comedy","Action,Adventure,Crime","Action,Adventure,Drama","Action,Adventure,Family","Action,Adventure,Fantasy","Action,Adventure,Horror",...,Sci-Fi,"Sci-Fi,Short",Short,Sport,Thriller,Western,director_id,writer_id,features,bert_encode
ml_id,,,,,,,,,,,,,,,,,,,,,
675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,270,1315,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.047421787, -0.2608599, -0.24956113, 0.09984..."
1542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,947,751,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.19441572, 0.08295966, -0.172608, 0.1050209..."
617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1018,970,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.2852734, 0.14134943, -0.22825861, 0.095383..."
656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,66,1382,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.5075418, 0.05868933, -0.037168838, -0.1534..."
1124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,373,688,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.21586643, 0.035775974, 0.06126339, 0.16846..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1107,1487,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.22981098, -0.034017388, 0.028036736, -0.05..."
1011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1107,1487,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.08646635, 0.088646404, -0.13403931, -0.001..."
751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1107,1487,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.20277898, 0.22795558, 0.073473245, 0.02763..."


In [202]:
print(imdb_features['bert_encode'].values[0])

[0.047421787, -0.2608599, -0.24956113, 0.0998459, -0.045840662, -0.24796191, 0.02909345, 0.26089373, 0.030790469, 0.19985321, -0.1253444, 0.3328539, 0.33847198, 0.6286125, 0.41038358, 0.17592657, -0.7269296, 0.5025794, 0.18513627, -0.2264853, -0.090477474, -0.4406627, 0.20858407, -0.21321052, 0.04375666, 0.078873105, 0.081799865, -0.279741, 0.06767028, 0.2120838, -0.40231466, 0.1717329, -0.09822548, -0.29655537, 0.34812725, -0.41114917, -0.064555414, -0.08552193, -0.21229729, -0.20822273, 0.235253, 0.03127867, 0.2934028, -0.118563384, 0.21157087, 0.021389881, -2.359247, -0.19583069, -0.14873251, 0.137159, 0.4590939, -0.06901635, 0.33159807, -0.005319435, 0.029741803, 0.3183286, -0.38151744, 0.41296816, 0.40303954, 0.108800024, 0.2464296, -0.014877507, -0.216464, -0.33298892, -0.07608058, 0.06549277, -0.029811254, 0.393121, -0.33336622, 0.53308505, -0.029374056, 0.0014352301, 0.046777975, -0.17739594, 0.16375697, -0.0989429, 0.098634504, -0.038493484, 0.058061663, 0.21430403, 0.14344299

In [203]:
imdb_features['features'] = imdb_features['features'].apply(lambda x: x.tolist())
print(imdb_features['features'].values[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 270, 1315]


In [204]:
imdb_features['final_features'] = imdb_features.apply(lambda x: x['features'] + x['bert_encode'], axis = 1)
print(imdb_features['final_features'].values[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 270, 1315, 0.047421787, -0.2608599, -0.24956113, 0.0998459, -0.045840662, -0.24796191, 0.02909345, 0.26089373, 0.030790469, 0.19985321, -0

/tmp/ipykernel_648409/1526203023.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  imdb_features['final_features'] = imdb_features.apply(lambda x: x['features'] + x['bert_encode'], axis = 1)


In [205]:
print(imdb_features['bert_encode'].apply(lambda x: len(x)).value_counts())
print(imdb_features['features'].apply(lambda x: len(x)).value_counts())
print(imdb_features['final_features'].apply(lambda x: len(x)).value_counts())
print(imdb_features['final_features'].apply(lambda x: type(x)).value_counts())

bert_encode
768    1656
Name: count, dtype: int64
features
289    1656
Name: count, dtype: int64
final_features
1057    1656
Name: count, dtype: int64
final_features
<class 'list'>    1656
Name: count, dtype: int64


In [206]:
# imdb_features['final_features'] = imdb_features['final_features'].apply(lambda x: torch.tensor(x))
# print(imdb_features['final_features'].apply(lambda x: type(x)).value_counts())

In [207]:
# np.min(imdb_features.index)

In [208]:
### imdb features를 ml_id를 기준으로 정렬
imdb_features = imdb_features.sort_index()
imdb_features

,Action,"Action,Adventure","Action,Adventure,Animation","Action,Adventure,Biography","Action,Adventure,Comedy","Action,Adventure,Crime","Action,Adventure,Drama","Action,Adventure,Family","Action,Adventure,Fantasy","Action,Adventure,Horror",...,"Sci-Fi,Short",Short,Sport,Thriller,Western,director_id,writer_id,features,bert_encode,final_features
ml_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,281,238,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.4952245, -0.2340898, -0.2483754, 0.0208517...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1107,1489,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.2663994, -0.05773397, 0.051528484, 0.01441...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,325,283,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.2685541, 0.1351281, -0.09522885, -0.126647...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,215,175,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.056043047, 0.02434545, 0.01950654, 0.10295...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,96,362,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.97943485, -0.44106466, -0.5316234, -0.5503...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1053,572,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.29111686, 0.049748316, -0.15580872, 0.0477...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,955,294,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.41580495, -0.0447725, -0.20376278, -0.0048...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1679,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,853,529,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.41111627, -0.11858723, -0.5411301, 0.03958...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [209]:
imdb_features = imdb_features[['final_features']]

In [210]:
imdb_features = imdb_features.reindex(range(1, np.max(imdb_features.index)+1), fill_value = None)

In [211]:
for idx, row in imdb_features.iterrows():
    if row['final_features'] is np.nan:
        imdb_features.loc[idx, 'final_features'] = [0]*1057

In [212]:
imdb_features

,final_features
ml_id,
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
1678,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1679,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1680,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [213]:
print(imdb_features['final_features'].apply(lambda x: len(x)).value_counts())

final_features
1057    1682
Name: count, dtype: int64


In [214]:
# imdb_features.to_csv('imdb_features.csv')

In [215]:
## mOVIELENS 아이템 개수
# n_m

In [216]:
print('train rating matrix shape: ', train_r.shape)
print('test rating matrix shape: ',test_r.shape)
print('imdb feature shape : ', imdb_features.shape)

train rating matrix shape:  (1682, 943)
test rating matrix shape:  (1682, 943)
imdb feature shape :  (1682, 1)


# Network Functions

In [217]:
def local_kernel(u, v):
    dist = torch.norm(u - v, p=2, dim=2)
    hat = torch.clamp(1. - dist**2, min=0.)
    
    return hat

# def local_kernel_imdb(u, v, imdb_features):
#     """
#     u: (n_in, 1, n_dim) - 입력 가중치
#     v: (1, n_hid, n_dim) - 은닉 가중치
#     imdb_features: (n_in, n_imdb_dim) - IMDb 피처 데이터
#     """
#     ## 기존 rating 기반 local kernel
#     dist = torch.norm(u - v, p=2, dim=2)
#     hat = torch.clamp(1. - dist**2, min=0.) # (n_in, n_hid)

#     ## IMDb 피처 기반 local kernel
#     imdb_tensor = torch.tensor(imdb_features.values, dtype=torch.float32).to(device)
#     imdb_similarity = torch.matmul(imdb_tensor, imdb_tensor.T) # Cosine similarity
#     print('matmul similarity: ', imdb_similarity.shape) # (n_in, n_in)
#     imdb_similarity = imdb_similarity.unsqueeze(2).expand(-1, -1, n_hid)  # (n_in, n_in, n_hid)
#     imdb_similarity_combined = torch.sum(imdb_similarity, dim=1, keepdim=True)  # (1682, 1)
#     # imdb_similarity_combined = imdb_similarity_combined.expand(-1, hat.size(1))  # (1682, 943)
#     weighted_hat = hat * imdb_similarity_combined
#     return weighted_hat
  

class KernelLayer(nn.Module):
    def __init__(self, n_in, n_hid, n_dim, lambda_s, lambda_2, activation=nn.Sigmoid()):
      super().__init__()
      print(n_in, n_hid, n_dim)
      self.W = nn.Parameter(torch.randn(n_in, n_hid))
      self.u = nn.Parameter(torch.randn(n_in, 1, n_dim))
      self.v = nn.Parameter(torch.randn(1, n_hid, n_dim))
      self.b = nn.Parameter(torch.randn(n_hid))

      self.lambda_s = lambda_s
      self.lambda_2 = lambda_2

      nn.init.xavier_uniform_(self.W, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.xavier_uniform_(self.u, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.xavier_uniform_(self.v, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.zeros_(self.b)
      self.activation = activation

    def forward(self, x):
      ### item information에 imdb 정보를 추가
      # imdb_features = torch.tensor(imdb_features.values, dtype=torch.float32).to(device)
      # imdb_features = imdb_features.unsqueeze(0).expand(u.size(0), -1, -1)
      # w_hat = local_kernel_imdb(self.u, self.v, imdb_features)
      w_hat = local_kernel(self.u, self.v)
      sparse_reg = torch.nn.functional.mse_loss(w_hat, torch.zeros_like(w_hat))
      sparse_reg_term = self.lambda_s * sparse_reg
      
      l2_reg = torch.nn.functional.mse_loss(self.W, torch.zeros_like(self.W))
      l2_reg_term = self.lambda_2 * l2_reg

      W_eff = self.W * w_hat  # Local kernelised weight matrix
      y = torch.matmul(x, W_eff) + self.b
      y = self.activation(y)

      return y, sparse_reg_term + l2_reg_term

class KernelNet(nn.Module):
    def __init__(self, n_u, n_hid, n_dim, n_layers, lambda_s, lambda_2):
      super().__init__()
      layers = []
      for i in range(n_layers):
        if i == 0:
          layers.append(KernelLayer(n_u, n_hid, n_dim, lambda_s, lambda_2))
        else:
          layers.append(KernelLayer(n_hid, n_hid, n_dim, lambda_s, lambda_2))
      layers.append(KernelLayer(n_hid, n_u, n_dim, lambda_s, lambda_2, activation=nn.Identity()))
      self.layers = nn.ModuleList(layers)
      self.dropout = nn.Dropout(0.33)

    def forward(self, x):
      total_reg = None
      for i, layer in enumerate(self.layers):
        x, reg = layer(x)
        if i < len(self.layers)-1:
          x = self.dropout(x)
        if total_reg is None:
          total_reg = reg
        else:
          total_reg += reg
      return x, total_reg

In [218]:
class CompleteNet(nn.Module):
    def __init__(self, kernel_net, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale):
      super().__init__()
      self.gk_size = gk_size
      self.dot_scale = dot_scale
      self.local_kernel_net = kernel_net
      self.conv_kernel = torch.nn.Parameter(torch.randn(n_m, gk_size**2) * 0.1)
      nn.init.xavier_uniform_(self.conv_kernel, gain=torch.nn.init.calculate_gain("relu"))
      

    def forward(self, x, x_local):
      gk = self.global_kernel(x_local, self.gk_size, self.dot_scale)
      x = self.global_conv(x, gk)
      x, global_reg_loss = self.local_kernel_net(x)
      return x, global_reg_loss

    def global_kernel(self, input, gk_size, dot_scale):
      avg_pooling = torch.mean(input, dim=1)  # Item (axis=1) based average pooling
      avg_pooling = avg_pooling.view(1, -1)

      gk = torch.matmul(avg_pooling, self.conv_kernel) * dot_scale  # Scaled dot product
      gk = gk.view(1, 1, gk_size, gk_size)

      return gk

    def global_conv(self, input, W):
      input = input.unsqueeze(0).unsqueeze(0)
      conv2d = nn.LeakyReLU()(F.conv2d(input, W, stride=1, padding=1))
      return conv2d.squeeze(0).squeeze(0)

class Loss(nn.Module):
    def forward(self, pred_p, reg_loss, train_m, train_r):
      # L2 loss
      diff = train_m * (train_r - pred_p)
      sqE = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff))
      loss_p = sqE + reg_loss
      return loss_p

# Network Instantiation

## Pre-training

In [219]:
n_in = n_u + len(imdb_features['final_features'].values[0])
# n_in = n_u + ml_onehot.shape[1]   #### 추가한 side information에 크기 맞춰줘야 함!!
model = KernelNet(n_in, n_hid, n_dim, n_layers, lambda_s, lambda_2).double().to(device)

2000 500 5
500 500 5
500 2000 5


## Fine-tuning

In [220]:
complete_model = CompleteNet(model, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale).double().to(device)

# Evaluation code

In [221]:
def dcg_k(score_label, k):
    dcg, i = 0., 0
    for s in score_label:
        if i < k:
            dcg += (2**s[1]-1) / np.log2(2+i)
            i += 1
    return dcg

In [222]:
def ndcg_k(y_hat, y, k):
    score_label = np.stack([y_hat, y], axis=1).tolist()
    score_label = sorted(score_label, key=lambda d:d[0], reverse=True)
    score_label_ = sorted(score_label, key=lambda d:d[1], reverse=True)
    norm, i = 0., 0
    for s in score_label_:
        if i < k:
            norm += (2**s[1]-1) / np.log2(2+i)
            i += 1
    dcg = dcg_k(score_label, k)
    return dcg / norm

In [223]:
def call_ndcg(y_hat, y):
    ndcg_sum, num = 0, 0
    y_hat, y = y_hat.T, y.T
    n_users = y.shape[0]

    for i in range(n_users):
        y_hat_i = y_hat[i][np.where(y[i])]
        y_i = y[i][np.where(y[i])]

        if y_i.shape[0] < 2:
            continue

        ndcg_sum += ndcg_k(y_hat_i, y_i, y_i.shape[0])  # user-wise calculation
        num += 1

    return ndcg_sum / num

# Training and Test Loop

In [224]:
best_rmse_ep, best_mae_ep, best_ndcg_ep = 0, 0, 0
best_rmse, best_mae, best_ndcg = float("inf"), float("inf"), 0

time_cumulative = 0
tic = time()

# Pre-Training
optimizer = torch.optim.AdamW(complete_model.local_kernel_net.parameters(), lr=0.001)

# def closure():
#   optimizer.zero_grad()
#   x = torch.Tensor(train_r).double().to(device)
#   m = torch.Tensor(train_m).double().to(device)
#   complete_model.local_kernel_net.train() #<<<<<<<<<<<<<<<<<<<<<<
#   pred, reg = complete_model.local_kernel_net(x)
#   loss = Loss().to(device)(pred, reg, m, x)
#   loss.backward()
#   return loss

def closure():
    #### IMDb 피처 결합해서 입력으로 주기기
    optimizer.zero_grad()

    # 기존 train_r와 IMDb 피처 결합
    x = torch.Tensor(train_r).double().to(device)  # (1682, 943)
    print('x shape: ', x.shape)
    side_tensor =  torch.Tensor(imdb_features['final_features'].tolist()).to(device)  # (1682, 289)
    # side_tensor = torch.Tensor(ml_onehot.values).double().to(device)  # (1682, 19)
    print('imdb_tensor shape: ', side_tensor.shape)
    x_combined = torch.cat((x, side_tensor), dim=1)  # (1682, 1232)

    # Mask와 모델 학습
    m = torch.Tensor(train_m).double().to(device)
    complete_model.local_kernel_net.train()
    pred, reg = complete_model.local_kernel_net(x_combined)  # 결합된 입력 사용

    # 사용자에 해당하는 출력만 비교
    pred_reduced = pred[:, :train_r.shape[1]]  # (1682, 943)
    loss = Loss().to(device)(pred_reduced, reg, m, x)
    loss.backward()
  
    return loss

last_rmse = np.inf
counter = 0

for i in range(max_epoch_p):
  optimizer.step(closure)
  complete_model.local_kernel_net.eval()
  t = time() - tic
  time_cumulative += t

  ############ 사이드 인포메이션에 맞게 입력데이터 바꿔주기 ###############
  train_tensor = torch.Tensor(train_r).double().to(device)  # (1682, 943)
  side_tensor =  torch.Tensor(imdb_features['final_features'].tolist()).to(device)  # (1682, 1082)
  # side_tensor = torch.Tensor(ml_onehot.values).double().to(device)  # (1682, 19)
  train_combined = torch.cat((train_tensor, side_tensor), dim=1)
  print('train_tensor shape: ', train_tensor.shape)
  print('side_tensor shape: ', side_tensor.shape)
  print('train_combined shape: ', train_combined.shape)

  

  # pre, _ = model(torch.Tensor(train_r).double().to(device))
  # pre = pre.float().cpu().detach().numpy()
  pre, _ = model(train_combined)
  pre_reduced = pre[:, :train_r.shape[1]]
  pre = pre_reduced.float().cpu().detach().numpy()
  
  error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()  # test error
  test_rmse = np.sqrt(error)

  error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
  train_rmse = np.sqrt(error_train)

  if last_rmse-train_rmse < tol_p:
    counter += 1
  else:
    counter = 0

  last_rmse = train_rmse

  if patience_p == counter:
    print('.-^-._' * 12)
    print('PRE-TRAINING')
    print('Epoch:', i+1, 'test rmse:', test_rmse, 'train rmse:', train_rmse)
    print('Time:', t, 'seconds')
    print('Time cumulative:', time_cumulative, 'seconds')
    print('.-^-._' * 12)
    break


  if i % 50 != 0:
    continue
  print('.-^-._' * 12)
  print('PRE-TRAINING')
  print('Epoch:', i, 'test rmse:', test_rmse, 'train rmse:', train_rmse)
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)


x shape:  torch.Size([1682, 943])
imdb_tensor shape:  torch.Size([1682, 1057])
train_tensor shape:  torch.Size([1682, 943])
side_tensor shape:  torch.Size([1682, 1057])
train_combined shape:  torch.Size([1682, 2000])
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 0 test rmse: 2.7744777 train rmse: 2.7535677
Time: 0.2663266658782959 seconds
Time cumulative: 0.2663266658782959 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
x shape:  torch.Size([1682, 943])
imdb_tensor shape:  torch.Size([1682, 1057])
train_tensor shape:  torch.Size([1682, 943])
side_tensor shape:  torch.Size([1682, 1057])
train_combined shape:  torch.Size([1682, 2000])
x shape:  torch.Size([1682, 943])
imdb_tensor shape:  torch.Size([1682, 1057])
train_tensor shape:  torch.Size([1682, 943])
side_tensor shape:  torch.Size([1682, 1057])
train_combined shape:  torch.Size([1682, 2000])
x shape:  torch.Size([1682, 943])
imdb_tensor shape:  torch.S

In [225]:
# Fine-Tuning
train_r_local = np.clip(pre, 1., 5.)

optimizer = torch.optim.AdamW(complete_model.parameters(), lr=0.001)

def closure():
    optimizer.zero_grad()

    ############ 사이드 인포메이션에 맞게 입력데이터 바꿔주기 ###############
    train_tensor = torch.Tensor(train_r).double().to(device)  # (1682, 943)
    train_local_tensor = torch.Tensor(train_r_local).double().to(device)  # (1682, 943)
    side_tensor =  torch.Tensor(imdb_features['final_features'].tolist()).to(device)
    # side_tensor = torch.Tensor(imdb_features.values).double().to(device)  # (1682, 289)
    # side_tensor = torch.Tensor(ml_onehot.values).double().to(device)  # (1682, 19)
    

    # IMDb 피처 결합한 입력 데이터 생성
    x_combined = torch.cat((train_tensor, side_tensor), dim=1)  # (1682, 1232)
    x_local_combined = torch.cat((train_local_tensor, side_tensor), dim=1)  # (1682, 1232)
    #####################################################################

    # Mask와 모델 학습
    m = torch.Tensor(train_m).double().to(device)
    complete_model.train()
    pred, reg = complete_model(x_combined, x_local_combined)  # IMDb 결합 데이터 사용
    pred_reduced = pred[:, :train_r.shape[1]]  # (1682, 943)
    loss = Loss().to(device)(pred_reduced, reg, m, train_tensor)  # IMDb 제외된 원래 평점과 비교
    loss.backward()

    return loss

last_rmse = np.inf
counter = 0

for i in range(max_epoch_f):
    optimizer.step(closure)
    complete_model.eval()
    t = time() - tic
    time_cumulative += t

    ############ 사이드 인포메이션에 맞게 입력데이터 바꿔주기 ###############
    train_tensor = torch.Tensor(train_r).double().to(device)  # (1682, 943)
    train_local_tensor = torch.Tensor(train_r_local).double().to(device)  # (1682, 943)
    side_tensor =  torch.Tensor(imdb_features['final_features'].tolist()).to(device)
    # side_tensor = torch.Tensor(imdb_features.values).double().to(device)  # (1682, 289)
    # side_tensor = torch.Tensor(ml_onehot.values).double().to(device)  # (1682, 19)
    

    # IMDb 피처 결합한 입력 데이터 생성
    x_combined = torch.cat((train_tensor, side_tensor), dim=1)  # (1682, 1232)
    x_local_combined = torch.cat((train_local_tensor, side_tensor), dim=1)  # (1682, 1232)
    #####################################################################

    # 모델 예측
    pre, _ = complete_model(x_combined, x_local_combined)
    pre_reduced = pre[:, :train_r.shape[1]]  # (1682, 943)
    pre = pre_reduced.float().cpu().detach().numpy()

    # Test RMSE 계산
    error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()
    test_rmse = np.sqrt(error)

    # Train RMSE 계산
    error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()
    train_rmse = np.sqrt(error_train)

    # Additional metrics
    test_mae = (test_m * np.abs(np.clip(pre, 1., 5.) - test_r)).sum() / test_m.sum()
    train_mae = (train_m * np.abs(np.clip(pre, 1., 5.) - train_r)).sum() / train_m.sum()

    test_ndcg = call_ndcg(np.clip(pre, 1., 5.), test_r)
    train_ndcg = call_ndcg(np.clip(pre, 1., 5.), train_r)

    # Update best metrics
    if test_rmse < best_rmse:
        best_rmse = test_rmse
        best_rmse_ep = i + 1

    if test_mae < best_mae:
        best_mae = test_mae
        best_mae_ep = i + 1

    if best_ndcg < test_ndcg:
        best_ndcg = test_ndcg
        best_ndcg_ep = i + 1

    # Early stopping
    if last_rmse - train_rmse < tol_f:
        counter += 1
    else:
        counter = 0

    last_rmse = train_rmse

    if patience_f == counter:
        print('.-^-._' * 12)
        print('FINE-TUNING')
        print('Epoch:', i + 1, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
        print('Epoch:', i + 1, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
        print('Time:', t, 'seconds')
        print('Time cumulative:', time_cumulative, 'seconds')
        print('.-^-._' * 12)
        break

    if i % 50 != 0:
        continue

    print('.-^-._' * 12)
    print('FINE-TUNING')
    print('Epoch:', i, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
    print('Epoch:', i, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
    print('Time:', t, 'seconds')
    print('Time cumulative:', time_cumulative, 'seconds')
    print('.-^-._' * 12)


.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 0 test rmse: 1.0832758 test mae: 0.86684084 test ndcg: 0.8249035482854342
Epoch: 0 train rmse: 1.0450343 train mae: 0.8379718 train ndcg: 0.8221443249631698
Time: 12.15491509437561 seconds
Time cumulative: 188.7052800655365 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 50 test rmse: 1.0226054 test mae: 0.81691873 test ndcg: 0.8601564299071746
Epoch: 50 train rmse: 0.9857071 train mae: 0.7878164 train ndcg: 0.8616747847583893
Time: 49.862720251083374 seconds
Time cumulative: 1756.8930265903473 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 100 test rmse: 1.0071188 test mae: 0.802098 test ndcg: 0.864722477469489
Epoch: 100 train rmse: 0.9703055 trai

In [226]:
# # Fine-Tuning

# train_r_local = np.clip(pre, 1., 5.)

# optimizer = torch.optim.AdamW(complete_model.parameters(), lr=0.001)

# def closure():
#   optimizer.zero_grad()
#   x = torch.Tensor(train_r).double().to(device)
#   x_local = torch.Tensor(train_r_local).double().to(device)
#   m = torch.Tensor(train_m).double().to(device)
#   complete_model.train()
#   pred, reg = complete_model(x, x_local)
#   loss = Loss().to(device)(pred, reg, m, x)
#   loss.backward()
#   return loss

# last_rmse = np.inf
# counter = 0

# for i in range(max_epoch_f):
#   optimizer.step(closure)
#   complete_model.eval()
#   t = time() - tic
#   time_cumulative += t

#   pre, _ = complete_model(torch.Tensor(train_r).double().to(device), torch.Tensor(train_r_local).double().to(device))
  
#   pre = pre.float().cpu().detach().numpy()

#   error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()  # test error
#   test_rmse = np.sqrt(error)

#   error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
#   train_rmse = np.sqrt(error_train)

#   test_mae = (test_m * np.abs(np.clip(pre, 1., 5.) - test_r)).sum() / test_m.sum()
#   train_mae = (train_m * np.abs(np.clip(pre, 1., 5.) - train_r)).sum() / train_m.sum()

#   test_ndcg = call_ndcg(np.clip(pre, 1., 5.), test_r)
#   train_ndcg = call_ndcg(np.clip(pre, 1., 5.), train_r)

#   if test_rmse < best_rmse:
#       best_rmse = test_rmse
#       best_rmse_ep = i+1

#   if test_mae < best_mae:
#       best_mae = test_mae
#       best_mae_ep = i+1

#   if best_ndcg < test_ndcg:
#       best_ndcg = test_ndcg
#       best_ndcg_ep = i+1

#   if last_rmse-train_rmse < tol_f:
#     counter += 1
#   else:
#     counter = 0

#   last_rmse = train_rmse

#   if patience_f == counter:
#     print('.-^-._' * 12)
#     print('FINE-TUNING')
#     print('Epoch:', i+1, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
#     print('Epoch:', i+1, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
#     print('Time:', t, 'seconds')
#     print('Time cumulative:', time_cumulative, 'seconds')
#     print('.-^-._' * 12)
#     break


#   if i % 50 != 0:
#     continue

#   print('.-^-._' * 12)
#   print('FINE-TUNING')
#   print('Epoch:', i, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
#   print('Epoch:', i, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
#   print('Time:', t, 'seconds')
#   print('Time cumulative:', time_cumulative, 'seconds')
#   print('.-^-._' * 12)

In [227]:
# Final result
print('Epoch:', best_rmse_ep, ' best rmse:', best_rmse)
print('Epoch:', best_mae_ep, ' best mae:', best_mae)
print('Epoch:', best_ndcg_ep, ' best ndcg:', best_ndcg)

Epoch: 1093  best rmse: 0.90484107
Epoch: 1374  best mae: 0.7098818
Epoch: 1062  best ndcg: 0.9005731175523306
